In [14]:
import DTAG_model_EF as dtagm
from importlib import reload
reload(dtagm)
import random
import pickle

from IPython.display import clear_output

In [2]:
import pandas as pd
import seaborn as sns
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from PIL import Image
from scipy.stats import iqr
%matplotlib inline

In [3]:
def gini(array):
    """Calculate the Gini coefficient of a numpy array."""
    #https://github.com/oliviaguest/gini/blob/master/gini.py
    # based on bottom eq:
    # http://www.statsdirect.com/help/generatedimages/equations/equation154.svg
    # from:
    # http://www.statsdirect.com/help/default.htm#nonparametric_methods/gini.htm
    # All values are treated equally, arrays must be 1d:
    array = array.flatten()
    if np.amin(array) < 0:
        # Values cannot be negative:
        array -= np.amin(array)
    # Values cannot be 0:
    array += 0.0000001
    # Values must be sorted:
    array = np.sort(array)
    # Index per array element:
    index = np.arange(1,array.shape[0]+1)
    # Number of array elements:
    n = array.shape[0]
    # Gini coefficient:
    return ((np.sum((2 * index - n  - 1) * array)) / (n * np.sum(array)))

def calculate_gini(CoupledModel):
    district_df = pd.DataFrame(CoupledModel.profit_district_end)
    no_district = len(district_df.columns)
    district_df['gini'] = district_df.apply(lambda x: gini(np.array(x[:no_district])), axis=1)
    return district_df['gini']

## No policy

In [6]:
reload(dtagm)

total_prod = {}
temporal_prod = {}
gini_coeffs = {}
profit_district_avrg = {}
profit_district_avrg_not_normalized = {}
profit_district_temporal = {}

for clim in [1,2]:
    for lud in [1,2]:
        for sed in [1,3,5]:
            try:
                del CoupledModel
            except:
                pass
            
            CoupledModel = dtagm.MetronamicaCoupled(#general setup of the model
                                                    start_yr = 2012, 
                                                    end_yr = 2050,
                                                    lu_map = 'Data//DTAG_NIAESLU2002_200m_v03a_wgs84utm48n_t05.tif',
                                                    geoproj = 'model_v03a_baj_process_v03_calib08a.geoproj',
                                                    log_option = 'cmd_log.xml',
                                                    fert_map = 'Data//DTAG_fertilizer.tif',
                                                    soil_map = 'Data//soil_DTAG_HWSD_full_recategorized.tif',
                                                    districts_map = 'Data//DTAG_districts_recoded.tif',
                                                    river_acc_map = 'Data//river_accessibility.tif',
                                                    sediment_rate_map='Data//fig7_interpolation_distance1.tif',
                                                    flood_coeff_map_all='Data//floodsim//all_s1_coeffs_v01.tif', 
                                                    flood_intercept_map_all='Data//floodsim//all_s1_intercepts_v01.tif',
                                                    flood_coeff_map_jul='Data//floodsim//jul_s1_coeffs_v01.tif', 
                                                    flood_intercept_map_jul='Data//floodsim//jul_s1_intercepts_v01.tif',    

                                                    #uncertainties parameters
                                                    clim_scen=clim, #1: rcp4.5, 2: rcp8.5
                                                    lud_scen=lud, #1: triple rice, 2: double rice
                                                    TE=sed,                                        

                                                    #policy parameters
                                                    pol_fer=0,
                                                    pol_discharge=0,
                                                    pol_seed=0,
                                                    p=False)
            
            CoupledModel.run()
            clear_output()
            
            scenario = 'run_clim'+str(clim)+'_lud'+str(lud)+'_TE'+str(sed)
            total_prod[scenario] = CoupledModel.total_rice_production
            temporal_prod[scenario] = CoupledModel.annual_rice_production
            gini_coeffs[scenario] = calculate_gini(CoupledModel)
            profit_district_avrg[scenario] = CoupledModel.average_profit_district
            profit_district_avrg_not_normalized[scenario] = CoupledModel.average_profit_district_not_normalized
            profit_district_temporal[scenario] = CoupledModel.profit_district_end

In [7]:
results_NoPol = {'total_prod': total_prod,
                 'temporal_prod': temporal_prod,
                 'gini_coeffs': gini_coeffs,
                 'profit_district_avrg:': profit_district_avrg,
                 'profit_district_avrg_not_normalized':profit_district_avrg_not_normalized,
                 'profit_district_temporal': profit_district_temporal}

fn = 'results/results_NoPol.pkl'
with open(fn, 'wb') as output:
    pickle.dump(results_NoPol, output, pickle.HIGHEST_PROTOCOL)

## High dikes policy

In [8]:
reload(dtagm)

total_prod = {}
temporal_prod = {}
gini_coeffs = {}
profit_district_avrg = {}
profit_district_avrg_not_normalized = {}
profit_district_temporal = {}

for clim in [1,2]:
    for lud in [1,2]:
        for sed in [1,3,5]:
            
            try:
                del CoupledModel_HD
            except:
                pass
            
            CoupledModel_HD = dtagm.MetronamicaCoupled(#general setup of the model
                                                    start_yr = 2012, 
                                                    end_yr = 2050,
                                                    lu_map = 'Data//DTAG_NIAESLU2002_200m_v03a_wgs84utm48n_t05.tif',
                                                    geoproj = 'model_v03a_baj_process_v03_calib08a.geoproj',
                                                    log_option = 'cmd_log.xml',
                                                    fert_map = 'Data//DTAG_fertilizer.tif',
                                                    soil_map = 'Data//soil_DTAG_HWSD_full_recategorized.tif',
                                                    districts_map = 'Data//DTAG_districts_recoded.tif',
                                                    river_acc_map = 'Data//river_accessibility.tif',
                                                    sediment_rate_map='Data//fig7_interpolation_distance1.tif',
                                                    flood_coeff_map_all='Data//floodsim//all_s1_coeffs_v01.tif', 
                                                    flood_intercept_map_all='Data//floodsim//all_s1_intercepts_v01.tif',
                                                    flood_coeff_map_jul='Data//floodsim//jul_s1_coeffs_v01.tif', 
                                                    flood_intercept_map_jul='Data//floodsim//jul_s1_intercepts_v01.tif',    

                                                    #uncertainties parameters
                                                    clim_scen=clim, #1: rcp4.5, 2: rcp8.5
                                                    lud_scen=lud, #1: triple rice, 2: double rice
                                                    TE=sed,                                        

                                                    #policy parameters
                                                    pol_fer=0,
                                                    pol_discharge=0,
                                                    pol_seed=0,
                                                    pol_dike_switch=1,
                                                    p=False)
            
            CoupledModel_HD.run()
            clear_output()
            
            scenario = 'run_clim'+str(clim)+'_lud'+str(lud)+'_TE'+str(sed)
            total_prod[scenario] = CoupledModel_HD.total_rice_production
            temporal_prod[scenario] = CoupledModel_HD.annual_rice_production
            gini_coeffs[scenario] = calculate_gini(CoupledModel_HD)
            profit_district_avrg[scenario] = CoupledModel_HD.average_profit_district
            profit_district_avrg_not_normalized[scenario] = CoupledModel_HD.average_profit_district_not_normalized
            profit_district_temporal[scenario] = CoupledModel_HD.profit_district_end

In [9]:
results_HighDikes = {'total_prod': total_prod,
                     'temporal_prod': temporal_prod,
                     'gini_coeffs': gini_coeffs,
                     'profit_district_avrg:': profit_district_avrg,
                     'profit_district_avrg_not_normalized':profit_district_avrg_not_normalized,
                     'profit_district_temporal': profit_district_temporal}

fn = 'results/results_HighDikes.pkl'
with open(fn, 'wb') as output:
    pickle.dump(results_HighDikes, output, pickle.HIGHEST_PROTOCOL)

## Low dikes policy

In [10]:
reload(dtagm)

total_prod = {}
temporal_prod = {}
gini_coeffs = {}
profit_district_avrg = {}
profit_district_avrg_not_normalized = {}
profit_district_temporal = {}

for clim in [1,2]:
    for lud in [1,2]:
        for sed in [1,3,5]:
            
            try:
                del CoupledModel_LD
            except:
                pass
            
            CoupledModel_LD = dtagm.MetronamicaCoupled(#general setup of the model
                                                    start_yr = 2012, 
                                                    end_yr = 2050,
                                                    lu_map = 'Data//DTAG_NIAESLU2002_200m_v03a_wgs84utm48n_t05.tif',
                                                    geoproj = 'model_v03a_baj_process_v03_calib08a.geoproj',
                                                    log_option = 'cmd_log.xml',
                                                    fert_map = 'Data//DTAG_fertilizer.tif',
                                                    soil_map = 'Data//soil_DTAG_HWSD_full_recategorized.tif',
                                                    districts_map = 'Data//DTAG_districts_recoded.tif',
                                                    river_acc_map = 'Data//river_accessibility.tif',
                                                    sediment_rate_map='Data//fig7_interpolation_distance1.tif',
                                                    flood_coeff_map_all='Data//floodsim//all_s1_coeffs_v01.tif', 
                                                    flood_intercept_map_all='Data//floodsim//all_s1_intercepts_v01.tif',
                                                    flood_coeff_map_jul='Data//floodsim//jul_s1_coeffs_v01.tif', 
                                                    flood_intercept_map_jul='Data//floodsim//jul_s1_intercepts_v01.tif',    

                                                    #uncertainties parameters
                                                    clim_scen=clim, #1: rcp4.5, 2: rcp8.5
                                                    lud_scen=lud, #1: triple rice, 2: double rice
                                                    TE=sed,                                        

                                                    #policy parameters
                                                    pol_fer=0,
                                                    pol_discharge=0,
                                                    pol_seed=0,
                                                    pol_dike_switch=2,
                                                    p=False)
            
            CoupledModel_LD.run()
            clear_output()
            
            scenario = 'run_clim'+str(clim)+'_lud'+str(lud)+'_TE'+str(sed)
            total_prod[scenario] = CoupledModel_LD.total_rice_production
            temporal_prod[scenario] = CoupledModel_LD.annual_rice_production
            gini_coeffs[scenario] = calculate_gini(CoupledModel_LD)
            profit_district_avrg[scenario] = CoupledModel_LD.average_profit_district
            profit_district_avrg_not_normalized[scenario] = CoupledModel_LD.average_profit_district_not_normalized
            profit_district_temporal[scenario] = CoupledModel_LD.profit_district_end

In [11]:
results_LowDikes = {'total_prod': total_prod,
                    'temporal_prod': temporal_prod,
                    'gini_coeffs': gini_coeffs,
                    'profit_district_avrg:': profit_district_avrg,
                    'profit_district_avrg_not_normalized':profit_district_avrg_not_normalized,
                    'profit_district_temporal': profit_district_temporal}

fn = 'results/results_LowDikes.pkl'
with open(fn, 'wb') as output:
    pickle.dump(results_LowDikes, output, pickle.HIGHEST_PROTOCOL)

## Fertilizer Policy

In [12]:
reload(dtagm)

total_prod = {}
temporal_prod = {}
gini_coeffs = {}
profit_district_avrg = {}
profit_district_avrg_not_normalized = {}
profit_district_temporal = {}

for clim in [1,2]:
    for lud in [1,2]:
        for sed in [1,3,5]:
            
            try:
                del CoupledModel_F
            except:
                pass
            
            CoupledModel_F = dtagm.MetronamicaCoupled(#general setup of the model
                                                    start_yr = 2012, 
                                                    end_yr = 2050,
                                                    lu_map = 'Data//DTAG_NIAESLU2002_200m_v03a_wgs84utm48n_t05.tif',
                                                    geoproj = 'model_v03a_baj_process_v03_calib08a.geoproj',
                                                    log_option = 'cmd_log.xml',
                                                    fert_map = 'Data//DTAG_fertilizer.tif',
                                                    soil_map = 'Data//soil_DTAG_HWSD_full_recategorized.tif',
                                                    districts_map = 'Data//DTAG_districts_recoded.tif',
                                                    river_acc_map = 'Data//river_accessibility.tif',
                                                    sediment_rate_map='Data//fig7_interpolation_distance1.tif',
                                                    flood_coeff_map_all='Data//floodsim//all_s1_coeffs_v01.tif', 
                                                    flood_intercept_map_all='Data//floodsim//all_s1_intercepts_v01.tif',
                                                    flood_coeff_map_jul='Data//floodsim//jul_s1_coeffs_v01.tif', 
                                                    flood_intercept_map_jul='Data//floodsim//jul_s1_intercepts_v01.tif',    

                                                    #uncertainties parameters
                                                    clim_scen=clim, #1: rcp4.5, 2: rcp8.5
                                                    lud_scen=lud, #1: triple rice, 2: double rice
                                                    TE=sed,                                        

                                                    #policy parameters
                                                    pol_fer=3,
                                                    pol_discharge=0,
                                                    pol_seed=0,
                                                    p=False)
            
            CoupledModel_F.run()
            clear_output()
            
            scenario = 'run_clim'+str(clim)+'_lud'+str(lud)+'_TE'+str(sed)
            total_prod[scenario] = CoupledModel_F.total_rice_production
            temporal_prod[scenario] = CoupledModel_F.annual_rice_production
            gini_coeffs[scenario] = calculate_gini(CoupledModel_F)
            profit_district_avrg[scenario] = CoupledModel_F.average_profit_district
            profit_district_avrg_not_normalized[scenario] = CoupledModel_F.average_profit_district_not_normalized
            profit_district_temporal[scenario] = CoupledModel_F.profit_district_end

In [13]:
results_Fertilizer = {'total_prod': total_prod,
                      'temporal_prod': temporal_prod,
                      'gini_coeffs': gini_coeffs,
                      'profit_district_avrg:': profit_district_avrg,
                      'profit_district_avrg_not_normalized':profit_district_avrg_not_normalized,
                      'profit_district_temporal': profit_district_temporal}

fn = 'results/results_Fertilizer.pkl'
with open(fn, 'wb') as output:
    pickle.dump(results_Fertilizer, output, pickle.HIGHEST_PROTOCOL)